# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
#print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance the local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, we need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = {'class' : 'SimpleStrategy',
                        'replication_factor' : 1}"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




###### Query 1

In [8]:
createTable = """
    CREATE TABLE IF NOT EXISTS artist_song_by_session (
        sessionId INT
        , itemInSession INT
        , artist TEXT
        , song TEXT
        , length DECIMAL
        , PRIMARY KEY (
            (sessionId, itemInSession)
            , artist
        )
    )
"""

## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1 = """
    SELECT 
        artist
        , song
        , length
    FROM artist_song_by_session
    WHERE sessionId=338 
        AND itemInSession=4
"""

In [9]:
# drop table
dropTable = "DROP TABLE IF EXISTS artist_song_by_session"
try:
    session.execute(dropTable)
except Exception as e:
    print(e)
    
# create table
try:
    session.execute(createTable)
except Exception as e:
    print(e)

In [10]:
# read and insert data into table
file = 'event_datafile_new.csv'

query = '' # 'reset' query string variable
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_by_session (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),    # convert sessionId to int
                                int(line[3]),    # convert itemInSession to int
                                line[0],
                                line[9],
                                float(line[5]))) # convert length to float

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.length,)   

Faithless Music Matters (Mark Knight Dub) 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

##### Query 2

In [12]:
createTable = """
    CREATE TABLE IF NOT EXISTS user_session (
        userId INT
        , sessionId INT
        , itemInSession INT
        , artist TEXT
        , song TEXT
        , firstName TEXT
        , lastName TEXT
        , PRIMARY KEY (
            (userId, sessionId)
            , itemInSession
        )
    )
"""

## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = """
    SELECT
        artist
        , song
        , firstName
        , lastName
    FROM user_session
    WHERE userId=10
        AND sessionId=182
"""                    

In [13]:
# drop table
dropTable = "DROP TABLE IF EXISTS user_session"
try:
    session.execute(dropTable)
except Exception as e:
    print(e)
    
# create table
try:
    session.execute(createTable)
except Exception as e:
    print(e)

In [14]:
# read and insert data into table
file = 'event_datafile_new.csv'

query = '' # 'reset' query string variable
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]),   # convert userId to int
                                int(line[8]),    # convert sessionId to int
                                int(line[3]),    # convert itemInSession to int
                                line[0],
                                line[9],
                                line[1],
                                line[4])) 

In [15]:
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)   

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


##### Query 3

In [16]:
createTable = """
    CREATE TABLE IF NOT EXISTS user_song (
         song TEXT
        , userId INT
        , firstName TEXT
        , lastName TEXT
        , PRIMARY KEY (
            (song)
            , firstName, lastName
        )
    )
"""

## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = """
    SELECT
        firstName
        , lastName
    FROM user_song
    WHERE song='All Hands Against His Own'
"""                    

In [17]:
# drop table
dropTable = "DROP TABLE IF EXISTS user_song"
try:
    session.execute(dropTable)
except Exception as e:
    print(e)
    
# create table
try:
    session.execute(createTable)
except Exception as e:
    print(e)

In [18]:
# read and insert data into table
file = 'event_datafile_new.csv'

query = '' # 'reset' query string variable
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9],
                                int(line[10]),  # convert userId to int
                                line[1],
                                line[4])) 

In [19]:
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

for row in rows:
    print (row.firstname, row.lastname)   

Jacqueline Lynch
Sara Johnson
Tegan Levine


### Drop the tables before closing out the sessions

In [20]:
## Drop the table before closing out the sessions
dropTable = "DROP TABLE IF EXISTS artist_song_by_session"
try:
    session.execute(dropTable)
except Exception as e:
    print(e)
    
dropTable = "DROP TABLE IF EXISTS user_session"
try:
    session.execute(dropTable)
except Exception as e:
    print(e)
    
dropTable = "DROP TABLE IF EXISTS user_song"
try:
    session.execute(dropTable)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()